In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px

In [2]:
# Reading super covid dataset
super_covid = pd.read_csv('../../Team/STAGE1/superCovidDS.CSV')
super_covid.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22_x,2020-01-23_x,2020-01-24_x,2020-01-25_x,2020-01-26_x,2020-01-27_x,...,2023-01-08_y,2023-01-09_y,2023-01-10_y,2023-01-11_y,2023-01-12_y,2023-01-13_y,2023-01-14_y,2023-01-15_y,2023-01-16_y,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,230,230,230,230,230,230,230,230,230,55869
1,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,719,719,719,719,721,721,721,721,721,223234
2,1005,Barbour County,AL,1,0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,24686
3,1007,Bibb County,AL,1,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,22394
4,1009,Blount County,AL,1,0,0,0,0,0,0,...,260,260,260,260,261,261,261,261,261,57826


In [3]:
super_covid_column_names = list(super_covid.columns)
super_covid_column_names[super_covid_column_names.index('County Name')] = "County_Name"
super_covid.columns=super_covid_column_names

In [4]:
transformed_df = pd.DataFrame(columns=['Date','Week','countyFIPS','County_Name', 'State', 'StateFIPS', 'population', 'Cases', 'New_Cases', 'Deaths', 'New_Deaths'])
transformed_df.head()

,Date,Week,countyFIPS,County_Name,State,StateFIPS,population,Cases,New_Cases,Deaths,New_Deaths


In [5]:
start_date = dt.datetime(2022,6,1)
end_date = dt.datetime(2022,12,31)
date_series = pd.date_range(start_date, end_date, freq='d')
date_delta = dt.timedelta(days=1)
for date in date_series:
    data = []
    for _ , row in super_covid.iterrows():
        temp = [date, date.isocalendar().week, getattr(row, 'countyFIPS'), getattr(row, 'County_Name'),
                getattr(row, 'State'), getattr(row, 'StateFIPS'), getattr(row, 'population')]
        cases_column = date.strftime('%Y-%m-%d_x')
        temp.append(getattr(row, cases_column))
        temp.append(getattr(row, cases_column) - getattr(row, (date-date_delta).strftime('%Y-%m-%d_x')))
        deaths_column = date.strftime('%Y-%m-%d_y')
        temp.append(getattr(row, deaths_column))
        temp.append(getattr(row, deaths_column) - getattr(row, (date-date_delta).strftime('%Y-%m-%d_y')))
        data.append(temp)
    transformed_df = pd.concat([transformed_df, pd.DataFrame(data, columns=transformed_df.columns)])
transformed_df.head()

,Date,Week,countyFIPS,County_Name,State,StateFIPS,population,Cases,New_Cases,Deaths,New_Deaths
0,2022-06-01,22,1001,Autauga County,AL,1,55869,15969,6,216,0
1,2022-06-01,22,1003,Baldwin County,AL,1,223234,56580,68,683,0
2,2022-06-01,22,1005,Barbour County,AL,1,24686,5710,3,99,0
3,2022-06-01,22,1007,Bibb County,AL,1,22394,6508,8,105,0
4,2022-06-01,22,1009,Blount County,AL,1,57826,15077,4,244,0


In [6]:
transformed_df.shape

(672388, 11)

In [7]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672388 entries, 0 to 3141
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Date         672388 non-null  datetime64[ns]
 1   Week         672388 non-null  object        
 2   countyFIPS   672388 non-null  object        
 3   County_Name  672388 non-null  object        
 4   State        672388 non-null  object        
 5   StateFIPS    672388 non-null  object        
 6   population   672388 non-null  object        
 7   Cases        672388 non-null  object        
 8   New_Cases    672388 non-null  object        
 9   Deaths       672388 non-null  object        
 10  New_Deaths   672388 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 61.6+ MB


In [8]:
#Changing datatypes
transformed_df = transformed_df.astype({'population':int,'Cases':int,'New_Cases':int,'Deaths':int,'New_Deaths':int})

In [9]:
transformed_df[['population','Cases','New_Cases','Deaths','New_Deaths']].describe()

,population,Cases,New_Cases,Deaths,New_Deaths
count,6.723880e+05,6.723880e+05,672388.000000,672388.000000,672388.000000
mean,1.044683e+05,2.830715e+04,18.528699,308.897577,0.068233
std,3.334039e+05,9.683983e+04,816.352474,1006.922337,12.226447
min,8.600000e+01,0.000000e+00,-546013.000000,0.000000,-7980.000000
25%,1.090100e+04,2.761000e+03,0.000000,42.000000,0.000000
50%,2.572600e+04,6.981000e+03,0.000000,101.000000,0.000000
75%,6.809800e+04,1.876200e+04,0.000000,239.000000,0.000000
max,1.003911e+07,3.420119e+06,167919.000000,34356.000000,3162.000000


We can see negative numbers in the New Cases and Deaths which could be beacuse of data inconsistency. Let us verify the data for those rows

In [10]:
transformed_df.query('New_Cases < 0')

,Date,Week,countyFIPS,County_Name,State,StateFIPS,population,Cases,New_Cases,Deaths,New_Deaths
387,2022-06-01,22,13001,Appling County,GA,13,18386,3558,-1,128,0
389,2022-06-01,22,13005,Bacon County,GA,13,11164,2666,-3,78,0
391,2022-06-01,22,13009,Baldwin County,GA,13,44890,7347,-10,240,0
392,2022-06-01,22,13011,Banks County,GA,13,19234,3432,-9,94,0
393,2022-06-01,22,13013,Barrow County,GA,13,83240,19650,-56,257,0
...,...,...,...,...,...,...,...,...,...,...,...
1602,2022-12-30,52,30009,Carbon County,MT,30,10725,2418,-2,29,0
2178,2022-12-30,52,40095,Marshall County,OK,40,16931,2571,-2837,48,0
2380,2022-12-30,52,46039,Deuel County,SD,46,4351,1172,-1,12,0
2400,2022-12-30,52,46079,Lake County,SD,46,12797,2588,-2,28,0


Filtered the data for negative New Cases, The first row is for Appling County shows negative New_Cases Let us verify the data for couple of Counties and those dates.

Picking the below entries for analysis
Appling County with Date Jun 1st. 
Barrow County with Date Jun1st
Marshall County with Date Dec30
Carbon County with Date Dec30

In [11]:
super_covid.head()

,countyFIPS,County_Name,State,StateFIPS,2020-01-22_x,2020-01-23_x,2020-01-24_x,2020-01-25_x,2020-01-26_x,2020-01-27_x,...,2023-01-08_y,2023-01-09_y,2023-01-10_y,2023-01-11_y,2023-01-12_y,2023-01-13_y,2023-01-14_y,2023-01-15_y,2023-01-16_y,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,230,230,230,230,230,230,230,230,230,55869
1,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,719,719,719,719,721,721,721,721,721,223234
2,1005,Barbour County,AL,1,0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,24686
3,1007,Bibb County,AL,1,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,22394
4,1009,Blount County,AL,1,0,0,0,0,0,0,...,260,260,260,260,261,261,261,261,261,57826


In [12]:
# Fetching cases data for above selected counties and dates(along with before and after) from super_covid
countyIds = [13001, 13013, 30009, 40095]
columns = ['County_Name','2022-05-30_x','2022-05-31_x','2022-06-01_x','2022-06-02_x','2022-12-28_x','2022-12-29_x','2022-12-30_x','2022-12-31_x']
super_covid.query(f'countyFIPS in {countyIds}')[columns]

,County_Name,2022-05-30_x,2022-05-31_x,2022-06-01_x,2022-06-02_x,2022-12-28_x,2022-12-29_x,2022-12-30_x,2022-12-31_x
387,Appling County,3559,3559,3558,3558,3757,3757,3794,3794
393,Barrow County,19706,19706,19650,19650,22164,22164,22737,22737
1602,Carbon County,2101,2101,2101,2101,2420,2420,2418,2418
2178,Marshall County,4608,4608,4608,4608,5408,5408,2571,2571


In [17]:
#plotting series of cases for a county
px.line(transformed_df.query('countyFIPS==40095'),x='Date',y='Cases')

In [ ]:
states_list = list(transformed_df['State'].unique())

def get_week_range_string(weekNumber):
    week_start = dt.datetime.strptime(f'2022-W{weekNumber}-1', "%Y-W%W-%w")
    week_end = dt.datetime.strptime(f'2022-W{weekNumber}-0', "%Y-W%W-%w")
    start_date = dt.datetime(2022, 6, 1)
    end_date = dt.datetime(2022, 12, 31)
    output_format = '%b-%d'
    if week_start < start_date:
        week_start = start_date
    if week_end > end_date:
        week_end = end_date
    return ' to '.join([week_start.strftime(output_format), week_end.strftime(output_format)])

def aggregate_state_df(state):
    if state in states_list:
        State_Covid = transformed_df.query(f"State=='{state}'").copy()
        State_aggregate_df = State_Covid.groupby(by=['State','Date','Week']).sum().reset_index()
        State_aggregate_df.drop(columns=['countyFIPS', 'StateFIPS', 'County_Name'], inplace=True)
        aggregations = ['mean', 'median', pd.Series.mode]
        State_Covid_Statistics = State_aggregate_df.groupby(by='Week').agg({'New_Cases': aggregations, 'New_Deaths': aggregations}).reset_index()
        State_Covid_Statistics.columns = ['_'.join(col) for col in State_Covid_Statistics.columns.values]
        cols = list(State_Covid_Statistics.columns)
        cols[cols.index('Week_')] = 'Week_Number'
        State_Covid_Statistics.columns = cols
        State_Covid_Statistics['Week_Dates'] = State_Covid_Statistics['Week_Number'].apply(get_week_range_string)
        State_Covid_Statistics['State'] = state
        State_Covid_Statistics['Population'] = State_aggregate_df['population'].unique()[0] 
        return State_Covid_Statistics

In [ ]:
NC_Covid_Statistics = aggregate_state_df('NC')
NC_Covid_Statistics.head()

In [ ]:
px.line(NC_Covid_Statistics, x='Week_Dates', y='New_Cases_mean', title='Weekly Average of New Cases in NC from Jun-22 to Dec-22')

In [ ]:
px.line(NC_Covid_Statistics, x='Week_Dates', y='New_Deaths_mean', title='Weekly Average of New Deaths in NC from Jun-22 to Dec-22')

In [ ]:
CA_Covid_Statistics = aggregate_state_df('CA')
CA_Covid_Statistics.head()

In [ ]:
NY_Covid_Statistics = aggregate_state_df('NY')
NY_Covid_Statistics.head()

In [ ]:
WA_Covid_Statistics = aggregate_state_df('WA')
WA_Covid_Statistics.head()

In [ ]:
Four_states_covid_stats = pd.concat([CA_Covid_Statistics,NC_Covid_Statistics,NY_Covid_Statistics,WA_Covid_Statistics],axis=0)
Four_states_covid_stats.head()

In [ ]:
px.line(Four_states_covid_stats,x='Week_Dates',y='New_Cases_mean',color='State', title = 'Weekly Average New Cases from Jun22 to Dec22')

In [ ]:
px.line(Four_states_covid_stats,x='Week_Dates',y='New_Deaths_mean',color='State', title='Weekly Average New Deaths from Jun22 to Dec22')

In [ ]:
Four_states_covid_stats['normalized_cases_mean'] =  100 * Four_states_covid_stats['New_Cases_mean']/Four_states_covid_stats['Population']
Four_states_covid_stats['normalized_deaths_mean'] =  100 * Four_states_covid_stats['New_Deaths_mean']/Four_states_covid_stats['Population']
Four_states_covid_stats['cases_mean_per_1M_population'] =  Four_states_covid_stats['New_Cases_mean']/100
Four_states_covid_stats['deaths_mean_per_1M_population'] =  Four_states_covid_stats['New_Deaths_mean']/100

Four_states_covid_stats.head()

In [ ]:
px.line(Four_states_covid_stats,x='Week_Dates',y='normalized_cases_mean',color='State')

In [ ]:
px.line(Four_states_covid_stats,x='Week_Dates',y='normalized_deaths_mean',color='State')

In [ ]:
US_covid_statistics = pd.read_csv('../../Team/Stage 2/US_level_covid_statistics.CSV')
US_covid_statistics.head()

In [ ]:
US_covid_statistics['normalized_cases_mean'] =  100 * US_covid_statistics['New_Cases_mean']/10000
US_covid_statistics['normalized_deaths_mean'] =  100 * US_covid_statistics['New_Deaths_mean']/US_covid_statistics['Population']
US_covid_statistics.head()

In [ ]:
px.line(US_covid_statistics,x='Week_Dates',y='normalized_cases_mean')

In [ ]:
px.line(US_covid_statistics,x='Week_Dates',y='normalized_deaths_mean')

In [ ]:
#Identifying 3 counties within CA state of high cases and death rates
CA_covid = transformed_df.query("State=='CA'").reset_index().drop(columns=['index','State','StateFIPS'])
CA_covid['case_rate'] = CA_covid['New_Cases']/CA_covid['population']
try:
    CA_covid['death_rate'] = CA_covid['New_Deaths']/CA_covid['New_Cases']
except ZeroDivisionError:
    CA_covid['death_rate'] = 0
CA_covid.head()

In [ ]:
CA_covid.groupby(['County_Name','population','countyFIPS']).agg({'New_Cases': sum,'New_Deaths': sum, "case_rate": sum, "death_rate": sum}).reset_index().sort_values('New_Cases',ascending=False)

In [ ]:
px.line(US_covid_statistics,x='Week_Dates',y='normalized_deaths_mean') 